function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)

In [ ]:
!cp -r '/content/drive/My Drive/tfrecord/train' '/content'
# train 데이터 복사
!zip -s 0 /content/train/tf_record_train.zip --out train.zip
# 분할 압축 파일 합치기
!rm -rf /content/train
# 분할 압축 파일 제거
!unzip /content/train.zip
# 압축 해제
!rm /content/train.zip
# 합친 압축파일 제거

In [ ]:
# 위와 동일(Valid set)
!cp -r '/content/drive/My Drive/tfrecord/valid' '/content'
!zip -s 0 /content/valid/tf_record_val.zip --out val.zip
!rm -rf /content/valid
!unzip /content/val.zip
!rm /content/val.zip

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

# Tensorflow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, BatchNormalization, PReLU
from tensorflow.keras.layers import Flatten, Activation, Dense, GlobalAveragePooling2D, Softmax
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow_addons import optimizers

In [ ]:
train_tfrecord_path = '/content/tf_record_train.tfrecords'
valid_tfrecord_path = '/content/tf_record_val.tfrecords'


BUFFER_SIZE = 256
BATCH_SIZE = 64
NUM_CLASS = 1049


image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'randmark_id': tf.io.FixedLenFeature([], tf.int64),
    # 'id': tf.io.FixedLenFeature([], tf.string),
}


    
def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def map_func(target_record):
    img = target_record['image_raw']
    label = target_record['randmark_id']
    img = tf.image.decode_jpeg(img, channels=3)
    if tf.random.uniform([]) > 0.5:
        img = tf.image.flip_left_right(img)
    img = tf.cast(img, tf.float32)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def map_func_valid(target_record):
    img = target_record['image_raw']
    label = target_record['randmark_id']
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def prep_func(image, label):
    result_image = image / 255
    result_image = tf.image.resize(result_image, (270,480))
    onehot_label = tf.one_hot(label, depth=NUM_CLASS)
    return result_image, onehot_label

dataset = tf.data.TFRecordDataset(train_tfrecord_path, compression_type='GZIP')
dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, compression_type='GZIP')
valid_dataset = valid_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(map_func_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_accuracy", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=1e-7,
                        verbose=1)
model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장
        filepath="/content/drive/My Drive/landmark_EFNETB5_model_270_480_radam.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)


callbacks = [earlystop, learning_rate_reduction, model_check]
# callbacks = [earlystop, learning_rate_reduction]

In [ ]:
from tensorflow.keras.applications import MobileNetV2
mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(270,480,3))


for layer in mobilenet.layers:
    layer.trainable = True

for (i, layer) in enumerate(mobilenet.layers):
    print(str(i), layer.__class__.__name__, layer.trainable)

model = Sequential()
model.add(mobilenet)
model.add(GlobalAveragePooling2D())
model.add(Dense(1049))
model.add(BatchNormalization())
model.add(PReLU())
model.add(Softmax(dtype='float32', name='softmax'))
# model.add(Flatten())
# model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(1049, activation='softmax')) 
model.summary()

In [ ]:
from tensorflow.keras.applications import inception_resnet_v2

efnet = EfficientNetB5(weights='imagenet' ,include_top=False, input_shape=(270,480,3))
for layer in efnet.layers:
    layer.trainable = True

for (i, layer) in enumerate(efnet.layers):
    print(str(i), layer.__class__.__name__, layer.trainable)

model = Sequential()
model.add(efnet)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dense(1049))
model.add(PReLU())
model.add(Softmax(dtype='float32', name='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
                optimizer=optimizers.RectifiedAdam(learning_rate=0.0001),
                metrics=['accuracy'])

In [ ]:
history = model.fit(dataset,
            epochs=1000,  
            validation_data=valid_dataset,
            callbacks = callbacks,
    )

In [ ]:
class Modeling_process:
    
    config = {
        'image_size' : (270, 480),
        'buffer' : 256,
        'batch_size' : 32,
        'learning_rate' : 0.001,
        'activation' : Adam
        

    }


    def __init__(self, train_path, valid_path):
        self.train_path = train_path
        self.valid_path = valid_path


    def preprocessing_data(self, image_size=self.config['image_size'], batch_size=self.config['batch_size'], buffer=self.config['buffer']):
        if image_size is None:
            image_size = self.config['image_size']

        self.dataset = tf.data.TFRecordDataset(self.train_path, compression_type='GZIP')
        self.dataset = self.dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        self.dataset = self.dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        self.dataset = self.dataset.shuffle(buffer)
        self.dataset = self.dataset.batch(batch_size)
        self.dataset = self.dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        self.dataset = self.dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

        self.valid_dataset = tf.data.TFRecordDataset(self.valid_path, compression_type='GZIP')
        self.valid_dataset = self.valid_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        self.valid_dataset = self.valid_dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        self.valid_dataset = self.valid_dataset.batch(batch_size)
        self.valid_dataset = self.valid_dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        self.valid_dataset = self.valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    
    def pre_model(self, pre_model, input_shape=self.config['image_size'], include_top=False, learning_rate=self.config['learning_rate'], activation=self.config['activation']):
        self.pre_model = pre_model(weights='imagenet', include_top=False, input_shape=(image_size[0],image_size[1],3))
        self.model = Sequential()
        self.model.add(self.pre_model)
        self.model.add(Flatten())
        self.model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.5))
        self.model.add(Dense(1049, activation='softmax')) 
        
        
        self.model.compile(loss='categorical_crossentropy',
                    optimizer=activation(learning_rate=learning_rate),
                    metrics=['accuracy'])
        
        earlystop = EarlyStopping(patience=5)
        learning_rate_reduction=ReduceLROnPlateau(
                                monitor= "val_accuracy", 
                                patience = 1, 
                                factor = 0.5, 
                                min_lr=0.00001,
                                verbose=1)
        model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장
                filepath=f"/content/drive/My Drive/landmark_{pre_model}.h5", #모델 파일 경로
                monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
                save_best_only=True)


